In [83]:
import os
import pandas as pd
import requests as req
import json
import numpy as np
import re
from unidecode import unidecode
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler

MAX_NAME_SIZE = 50
DICT_SIZE = 26

In [17]:
DATA_URL = 'https://api.got.show/api/characters/'

data = req.get(DATA_URL).content

In [21]:
# OPTIONAL

with open(os.path.join('data', 'characters.json'), 'r') as f:
    data = f.read()

In [22]:
df = pd.read_json(data, orient='records')
len(df)

2028

In [23]:
df = df[['name', 'dateOfBirth', 'dateOfDeath']]

In [24]:
df_only_born = df.dropna(subset=['dateOfBirth'])
len(df_only_born)

478

In [84]:
def preprocess_name(name):
    out_name = unidecode(name.lower())
    out_name = re.sub(r'\W', '', out_name)
    
    if len(out_name) > MAX_NAME_SIZE:
        out_name = out_name[:MAX_NAME_SIZE]
    
    return out_name

def preprocess_age(birth, death):
    if np.isnan(death):
        return -1
    else:
        return int(death - birth)

In [26]:
df_names = df_only_born['name'].apply(preprocess_name)

In [31]:
list_ages = []

for birth, death in zip(list(df_only_born['dateOfBirth']), list(df_only_born['dateOfDeath'])):
    list_ages.append(preprocess_age(birth, death))

In [52]:
def encode_name(name):
    return one_hot(' '.join(list(name)), DICT_SIZE)

In [56]:
encoded_names = []

# encode names in one hot
for index, name in enumerate(list(df_names)):
    encoded_names.append(encode_name(name))

# pad sequences
encoded_names = pad_sequences(encoded_names, maxlen=MAX_NAME_SIZE, padding='post')
encoded_names = np.array(encoded_names)
encoded_names.shape

(478, 50)

In [57]:
mm_scaler = MinMaxScaler()
scaler = mm_scaler.fit([[age] for age in list_ages])

In [58]:
ages_scaled = scaler.transform([[age] for age in list_ages])
ages_scaled = np.array(ages_scaled)
ages_scaled.shape

(478, 1)

# Machine Learning

In [146]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten

model = Sequential()
model.add(Embedding(DICT_SIZE, 50, input_length=MAX_NAME_SIZE))
model.add(Flatten())
model.add(Dense(units=25, activation='sigmoid'))
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 50, 50)            1300      
_________________________________________________________________
flatten_6 (Flatten)          (None, 2500)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 25)                62525     
_________________________________________________________________
dense_20 (Dense)             (None, 10)                260       
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 64,096
Trainable params: 64,096
Non-trainable params: 0
_________________________________________________________________


In [147]:
model.compile(optimizer="adam", loss='binary_crossentropy',  metrics=['accuracy'])

history = model.fit(x=encoded_names, y=ages_scaled, epochs=200, verbose=1, shuffle=True)

#score = model.evaluate(x_test, y_test_oh, verbose=0)

#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

Epoch 1/200
478/478 [==============================] - 0s 713us/step - loss: 1.6501 - acc: 0.0021
Epoch 2/200
478/478 [==============================] - 0s 193us/step - loss: 1.1579 - acc: 0.0021
Epoch 3/200
478/478 [==============================] - 0s 171us/step - loss: 0.9146 - acc: 0.0021
Epoch 4/200
478/478 [==============================] - 0s 200us/step - loss: 0.7963 - acc: 0.0021
Epoch 5/200
478/478 [==============================] - 0s 198us/step - loss: 0.7175 - acc: 0.0921
Epoch 6/200
478/478 [==============================] - 0s 203us/step - loss: 0.6598 - acc: 0.6569
Epoch 7/200
478/478 [==============================] - 0s 169us/step - loss: 0.6181 - acc: 0.6569
Epoch 8/200
478/478 [==============================] - 0s 183us/step - loss: 0.5864 - acc: 0.6569
Epoch 9/200
478/478 [==============================] - 0s 195us/step - loss: 0.5618 - acc: 0.6569
Epoch 10/200
478/478 [==============================] - 0s 146us/step - loss: 0.5419 - acc: 0.6569
Epoch 11/200
478/47

478/478 [==============================] - 0s 147us/step - loss: 0.3683 - acc: 0.6569
Epoch 84/200
478/478 [==============================] - 0s 151us/step - loss: 0.3683 - acc: 0.6569
Epoch 85/200
478/478 [==============================] - 0s 149us/step - loss: 0.3682 - acc: 0.6569
Epoch 86/200
478/478 [==============================] - 0s 151us/step - loss: 0.3682 - acc: 0.6569
Epoch 87/200
478/478 [==============================] - 0s 179us/step - loss: 0.3682 - acc: 0.6569
Epoch 88/200
478/478 [==============================] - 0s 131us/step - loss: 0.3682 - acc: 0.6569
Epoch 89/200
478/478 [==============================] - 0s 138us/step - loss: 0.3681 - acc: 0.6569
Epoch 90/200
478/478 [==============================] - 0s 163us/step - loss: 0.3681 - acc: 0.6569
Epoch 91/200
478/478 [==============================] - 0s 150us/step - loss: 0.3681 - acc: 0.6569
Epoch 92/200
478/478 [==============================] - 0s 176us/step - loss: 0.3680 - acc: 0.6569
Epoch 93/200
478/478 [=

478/478 [==============================] - 0s 150us/step - loss: 0.3679 - acc: 0.6569
Epoch 166/200
478/478 [==============================] - 0s 154us/step - loss: 0.3679 - acc: 0.6569
Epoch 167/200
478/478 [==============================] - 0s 168us/step - loss: 0.3679 - acc: 0.6569
Epoch 168/200
478/478 [==============================] - 0s 141us/step - loss: 0.3679 - acc: 0.6569
Epoch 169/200
478/478 [==============================] - 0s 152us/step - loss: 0.3679 - acc: 0.6569
Epoch 170/200
478/478 [==============================] - 0s 134us/step - loss: 0.3679 - acc: 0.6569
Epoch 171/200
478/478 [==============================] - 0s 174us/step - loss: 0.3678 - acc: 0.6569
Epoch 172/200
478/478 [==============================] - 0s 150us/step - loss: 0.3678 - acc: 0.6569
Epoch 173/200
478/478 [==============================] - 0s 163us/step - loss: 0.3677 - acc: 0.6569
Epoch 174/200
478/478 [==============================] - 0s 161us/step - loss: 0.3678 - acc: 0.6569
Epoch 175/200


In [148]:
my_name = encode_name(preprocess_name('Vinicius Matheus Veríssimo da Silva'))
my_name = pad_sequences([my_name], maxlen=MAX_NAME_SIZE, padding='post')
my_name.shape

(1, 50)

In [149]:
res = model.predict(np.array(my_name))
print(res[0][0])
scaler.inverse_transform(res[0][0])

0.120866


array([[ 11.44918156]], dtype=float32)